# **1. Import Libraries**

In [ ]:
# Reloading modules automatically when they change
%load_ext autoreload
%autoreload 2

In [ ]:
from forecaster.data import spot
from forecaster.models import models, trainer
from forecaster.plotting.interactive import plot_predictions
from forecaster.data.fingrid import EXTERNAL_FEATURES

# **2. Load Data**

In [ ]:
import os

# Load the data
file_path = '../data/spot_prices_fi_2016_2023.csv'
spot_data = spot.load_spot_data(file_path, date_col="date", price_col="elspot-fi")
plot_predictions(spot_data)

# **3. Model Training**
Fit a linear model with LEAR features highly inspired by the following paper: [LEAR Paper](https://www.sciencedirect.com/science/article/pii/S0306261921004529?via%3Dihub)

In [ ]:
# model = models.LinearModel(daily_price_lags=[1])
# model = models.LinearModel(daily_price_lags=[1, 2, 3, 7], time_features=True)
model = models.LinearModel(daily_price_lags=[1, 2, 3, 7], time_features=True, efternal_features=EXTERNAL_FEATURES, daily_external_lags=[1, 7])
data_with_features = model.preprocess_data(spot_data)

predictions, metrics, coeffs = trainer.year_on_year_training(data_with_features, model)
print(metrics)
plot_predictions(spot_data, predictions)

## **4. Export Predictions**

In [ ]:
raw_spot_data = spot.load_spot_data(file_path, date_col="date", price_col="elspot-fi", raw=True)

# Determine the new file path
directory, full_file_name = os.path.split(file_path)
file_name, file_extension = os.path.splitext(full_file_name)
new_full_file_name = f"{file_name}_with_predictions{file_extension}"
new_file_path = os.path.join(directory, new_full_file_name)

# Merge the original spot data with model predictions, insert nan where predictions are missing
spot_data_and_predictions = raw_spot_data.merge(predictions['test'], left_index=True, right_index=True, how='left')
spot_data_and_predictions.rename(columns={'y_hat': 'predicted_price'}, inplace=True)

# Reset the index and just keep the original date column
spot_data_and_predictions.reset_index(drop=True, inplace=True)

# Save the predictions to a new file together with the original data
spot_data_and_predictions.to_csv(new_file_path)
spot_data_and_predictions.tail()